In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
selection_multi = ['multi']

trainingDataset = pd.read_json('./train.jsonl', lines=True)
trainingDataset.rename({'tags': 'label'}, axis=1, inplace=True)
trainingDataset['postText'] = trainingDataset.postText.apply(lambda x: x[0])
trainingDataset['targetParagraphs'] = trainingDataset.targetParagraphs.apply(lambda p: ''.join(p))

mask_multi_not_act = trainingDataset.label.apply(lambda x: not any(item for item in selection_multi if item in x))

trainingDatasetFiltered = trainingDataset[mask_multi_not_act]
trainingDatasetFiltered['label'] = trainingDataset.label.apply(lambda t: 0 if (t == ['phrase']) else 1)
features = Features({"uuid": Value("string"), "postId": Value("string"), "postText": Value("string"), "postPlatform": Value("string"), "targetParagraphs": Value("string"), "targetTitle": Value("string"), "targetDescription": Value("string"), "targetKeywords": Value("string"), "label": ClassLabel(num_classes=2, names=[0,1])})

relevant_columns = trainingDatasetFiltered[["uuid", "postId", "postText", "postPlatform", "targetParagraphs", "targetTitle", "targetDescription", "targetKeywords", "label"]]

huggingDatasetTrain = Dataset.from_pandas(relevant_columns[:1000], features=features)
huggingDatasetTest = Dataset.from_pandas(relevant_columns[1000:2000], features=features)

print(huggingDatasetTrain)

ds = DatasetDict()
ds['train'] = huggingDatasetTrain
ds['test'] = huggingDatasetTest


def tokenize_function(examples):
    return tokenizer(examples['postText'], examples['targetParagraphs'], padding="max_length", return_tensors="np", truncation=True)

tokenized_data = ds.map(tokenize_function, batched=True)

tokenized_train = tokenized_data['train']
tokenized_test = tokenized_data['test']

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer")

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", remove_unused_columns=True)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)


trainer.train()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Dataset({
    features: ['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs', 'targetTitle', 'targetDescription', 'targetKeywords', 'label'],
    num_rows: 1000
})


<ipython-input-1-2622b8fffdb6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingDatasetFiltered['label'] = trainingDataset.label.apply(lambda t: 0 if (t == ['phrase']) else 1)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.